# Creating Normalised Training Data

In this notebook we will be doing the following :
  1. Build & perform basic text cleaning operations/pipeline on the documents
  2. Convert the data in training data format. i.e. _label_tag1 _label_tag2
  3. Split the dataset into : train,dev and test sets  
  4.  Divide the dataset into Groups/Classes

    a. Dividing the entire Dataset(~1M) into 14 groups/class.

    b. Check the Label Distribution of Labels in each 14 groups/class.
    
    c. Set Threshold to clip the maximum number of examples of a Label in a group/class.

  5. Create Corpus & Label Dictionary : Flair Corpus

**HOT TIP** : *Save them as pickle for easy rendering for experiments*



In [0]:
# First let's check what has Google given us ! Thank you Google for the GPU
!nvidia-smi

Wed Dec 25 02:49:18 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [8]:
# Let's mount our G-Drive. Hey !! Because for GPU you now give your data to Google 

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Install necessary packages and restart the environment

! pip install tiny-tokenizer
! pip install  flair

In [0]:
# Let's import our packages !

import pandas as pd
from tqdm import tqdm
import html
import re
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split
# import flair
import pickle
from torch.optim.adam import Adam
import json

# Making Corpus

from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.samplers import ImbalancedClassificationDatasetSampler

In [0]:
## Mentioning where is our data located on G-Drive. Make sure to rectify your path

path = '/content/drive/My Drive/ICDMAI_Tutorial/notebook/'
tag_group = '/content/drive/My Drive/ICDMAI_Tutorial/stack-overflow-tag-network/stack_network_nodes.csv'
data ='filtered_data/question_tag_text_mapping.pkl'

In [0]:
# Let's see the main Data-Set

question_tag = pd.read_pickle(path+data)
question_tag.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,CreationMonth,CreationYear,Tag
0,120,83.0,2008-08-01 15:50:08,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,8,2008,"[asp.net, sql]"
1,260,91.0,2008-08-01 23:22:08,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,8,2008,"[c#, .net]"
2,330,63.0,2008-08-02 02:51:36,NaN,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,8,2008,[c++]
3,470,71.0,2008-08-02 15:11:47,2016-03-26T05:23:29Z,13,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,8,2008,"[web-services, .net]"
4,580,91.0,2008-08-02 23:30:59,NaN,21,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...,8,2008,[sql-server]


In [0]:
# Let's see the Groups/Classes

tag_group = pd.read_csv(tag_group)
tag_group.head()

,name,group,nodesize
0,html,6,272.45
1,css,6,341.17
2,hibernate,8,29.83
3,spring,8,52.84
4,ruby,3,70.14


## 1. Text Pre-processing Pipeline

Every try-except block can be written as a different modular function which can be invoked from preprocess_text() function. This serves as a pipeline of the series of text-cleaning that you might require for your dataset.

In [0]:
clean = re.compile('<.*?>')

def preprocess_text(text) :
  try :
    # soup = BeautifulSoup(text, "html.parser")
    # text = soup.get_text()
    text=  re.sub(clean, '', text)
    text = html.unescape(text)
  except :
    print("Error in HTML Processing ...")
    print(text)
    text = text
  try :
    # remove extra newlines (often might be present in really noisy text)
    text = text.translate(text.maketrans("\n\t\r", "   "))
  except :
    print("Error in removing extra lines ...")
    print(text)
    text = text

  try :
    # remove extra whitespace
    text = re.sub(' +', ' ', text)
    text = text.strip()
  except :
    print("Error in extra whitespace removal ...")
    print(text)
    text = text

  return text

## 2. Create Training Data Format

Here we iterate the dataset dataframe and create the format acceptable to Flair. This is a standard format for few other Text Classification models/frameworks by Facebook.

***Format***  : ____label ____**tag1** ____label ____**tag2** **text**

Here the text Document has to be in a single line which was handled in the preprocess_text() method.


In [0]:
def create_training_format(question_tag) :

  print("Preparing training data format ...")
  # training_df = pd.DataFrame("columns")
  labels = list()
  texts = list()
  for index in tqdm(question_tag.index) :
    tags = question_tag.loc[index,'Tag']
    text_label = ''
    for tag in tags :
      label = '__label__'+tag
      text_label = text_label + ' ' + label
    
    text_label = text_label.strip()
    # text =  html.unescape(question_tag.loc[index,'Body'])
    text =  question_tag.loc[index,'Title'].strip() + '. ' + question_tag.loc[index,'Body'].strip()

    # if len(text.split()) < 5 :
    #   continue 

    labels.append(text_label)
    texts.append(text)


  df = pd.DataFrame(list(zip(labels[:], texts[:])), columns =['label', 'text']) 
  # df.head()
  print("Cleaning Text ....")
  df['text'] = df['text'].apply(preprocess_text)
  print("Cleaned Data Size : {}".format(df.shape))

  return df

## 3. Create Training Splits

Here we create standard random splits of the dataset to :
  1. training set : 90 % data
  2. dev/validation set :  10 % data
  3. test set : 10 % data

#### TO DO : Experiments :
  1. Stratified Sampling
  2. Does a  70-15-15 split or 90-5-5 split make any difference when you ahve 1M records ?

In [0]:
def create_splits(df,path,group_id = ''):

  print("Splitting Training Data ... ")
  df = df.sample(frac=1)
  train_df , test_df = train_test_split(df,random_state=42,test_size=0.20)
  dev_df ,test_df = train_test_split(test_df,random_state=42,test_size=0.5)
  print("Training Dataset : {}".format(train_df.shape[0]))
  print("Validation Dataset : {}".format(dev_df.shape[0]))
  print("Test Dataset : {}".format(test_df.shape[0]))

  print("Path  : {} ".format(path+'training_data/normalised_training_data/group/'+ str(group_id) + '/train.txt'))
  train_df.to_csv(path+'training_data/normalised_training_data/group/'+ str(group_id) + '/train.txt',sep='\t',index=False,header=False)
  dev_df.to_csv(path+'training_data/normalised_training_data/group/'+ str(group_id) + '/dev.txt',sep='\t',index=False,header=False)
  test_df.to_csv(path+'training_data/normalised_training_data/group/'+ str(group_id) + '/test.txt',sep='\t',index=False,header=False)

  return train_df,dev_df,test_df

## 4. Divide the dataset into Groups/Classes

Here we iterate over the entire dataset to create group level datasets in the following steps :
  1. Iterate over the groups and read the full-dataset eveytime
  2. Get all the tags in the group from the **tag_group** lookup
  3. Iterate over training examples and see if the labels fall in the same group
  4. Remove training examples which don't belong to the group
  5. Se document with multiple labels and just one label
  6. We keep all the documents with more than one label to keep diversity
  7. With documents with 1 label will, balance the dataset by cliping the number of examples for the label
  8. Finally merge both the dataset.
  9. Create the training data format of the remaining dataset
  10. Split & Save the dataset




In [0]:
for grp_id in range(1,15):
  
  ## 1. Iterate over the groups and read the full-dataset eveytime
  print("=================================================================")
  print("Group ID being Processed : {}".format(grp_id))
  print("=================================================================")
  print("Reading Pickle File ...")
  question_tag = pd.read_pickle(path+data)

  ## 2. Get all the tags in the group from the **tag_group** lookup  

  group =  tag_group[tag_group.group == grp_id]
  labels = list(set(group['name']))

  ## 3. Iterate over training examples and see if the labels fall in the same group
  for index in tqdm(question_tag.index):
    tags = question_tag.loc[index,'Tag']
    group_tags = list()
    for tag in tags :
      if tag in labels :
        group_tags.append(tag)
    question_tag.at[index,'Tag'] =  group_tags

  print("Before Removal of Blank Data : {} ".format(question_tag.shape))

  ## 4. Remove training examples which don't belong to the group
  question_tag = question_tag[question_tag['Tag'].map(lambda d: len(d)) > 0]
  print("Final Data for Group ID  : {} is {}".format(grp_id,question_tag.shape))

  # Segregating Single & Multiple Labels

  question_tag['tag_count'] = question_tag['Tag'].apply(len)
  question_tag = question_tag.sort_values(by = ['tag_count'],ascending=False)

  print("Training Data Size : {}".format(question_tag.shape[0]))
  
  ## 5. Remove document with multiple labels and just one label
  multi_tags = question_tag[question_tag.tag_count >  1]
  single_tags = question_tag[question_tag.tag_count ==  1]

  ## 6. We keep all the documents with more than one label to keep diversity
  print("Size of Multi-Tag Documents :: {}".format(multi_tags.shape[0]))
  
  
  ## 7. With documents with 1 label will, balance the dataset by cliping the number of examples for the label
  # Adding Single tag outside list
  single_tags['open_tag'] = ''
  for index in single_tags.index :
    single_tags.at[index,'open_tag'] = single_tags.loc[index,'Tag'][0
                                                                    ]
  single_tag_grp = single_tags.groupby(['open_tag'],as_index=False).agg({'tag_count' :  'count'}).sort_values(by=['tag_count'],ascending=False)
  print("Decide On Cutt of for Single Threshold ...")
  print(single_tag_grp)

  # Clipping the number of examples via-user input
  threshold = int(input())

  final_df = pd.DataFrame()

  for open_tag in list(single_tag_grp['open_tag'] ) :
    open_tag_df = single_tags[single_tags.open_tag == open_tag].sample(frac=1).reset_index(drop=True).head(threshold)
    final_df = final_df.append(open_tag_df,ignore_index=False)

  print("Verify")
  print(final_df.groupby(['open_tag'],as_index=False).agg({'tag_count' :  'count'}).sort_values(by=['tag_count'],ascending=False))

  multi_tags.pop('tag_count')
  final_df.pop('tag_count')
  final_df.pop('open_tag')

  ## 8. Finally merge both the dataset.
  multi_tags= multi_tags.append(final_df,ignore_index=True)
  print("Training Data Size : {}".format(multi_tags.shape[0]))

  ## 9. Create the training data format of the remaining dataset
  training_data_format = create_training_format(multi_tags)

  ## 10. Split & Save the dataset
  train_df,dev_df,test_df = create_splits(training_data_format,path,group_id=grp_id)
  
  print("=================================================================")


Group ID being Processed : 1
Reading Pickle File ...


100%|██████████| 1051992/1051992 [00:46<00:00, 22611.41it/s]


Before Removal of Blank Data : (1051992, 10) 
Final Data for Group ID  : 1 is (163030, 10)
Training Data Size : 163030
Size of Multi-Tag Documents :: 15824


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Decide On Cutt of for Single Threshold ...
           open_tag  tag_count
7            python      55314
1               c++      41047
0                 c      19528
9                 r      15298
2            django       6475
6            matlab       5967
8                qt       2308
5  machine-learning        726
3          embedded        308
4             flask        235
6500


...

Decide On Cutt of for Single Threshold ...
    open_tag  tag_count
0      excel       5467
2        vba       2158
1  excel-vba        982
2200


 30%|███       | 3316/11024 [00:00<00:00, 33157.66it/s]

Verify
    open_tag  tag_count
0      excel       2200
2        vba       2158
1  excel-vba        982
Training Data Size : 11024
Preparing training data format ...


100%|██████████| 11024/11024 [00:00<00:00, 32211.78it/s]


Cleaning Text ....
Cleaned Data Size : (11024, 2)
Splitting Training Data ... 
Training Dataset : 8819
Validation Dataset : 1102
Test Dataset : 1103
Path  : /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/14/train.txt 


## 5. Create Corpus & Label Dictionary : Flair Corpus

For all the training splits created above for each group, we will be creating a corpus & vocabulary to train a different model.

In [0]:
 

path = '/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/'
for grp in range(1,15):
  print("=================================================================")
  print("Group ID being Processed : {}".format(grp))
  print("=================================================================")

  # print(path+str(grp))
  # this is the folder in which train, test and dev files reside
  data_folder =path+str(grp)+'/'
  print(data_folder)

  print("Creating Corpus ...")
  # load corpus containing training, test and dev data
  corpus: Corpus = ClassificationCorpus(data_folder,
                                        test_file='test.txt',
                                        dev_file='dev.txt',
                                        train_file='train.txt')
                                        # max_tokens_per_doc = 100,
                                        # tokenizer = True,
                                        # in_memory = False)

  print("Creating Label Dictionary ...")
  # 2. create the label dictionary
  label_dict = corpus.make_label_dictionary()

  print("Obtaining Corpus Statisitics...")
  stats  = corpus.obtain_statistics()
  json_stats = json.loads(stats)

  print(json_stats)
  
  with open(data_folder+'corpus_statistics.json', 'w') as f:
    json.dump(json_stats, f)

  print("Creating Dumps ... ")
  with open(data_folder+'classification_corpus.pkl',mode='wb') as f :
    pickle.dump(corpus,f)

  with open(data_folder + 'classification_corpus_label_dict.pkl',mode='wb') as f:
    pickle.dump(label_dict,f)

Group ID being Processed : 1
/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/1/
Creating Corpus ...
2019-12-24 18:38:01,633 Reading data from /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/1
2019-12-24 18:38:01,635 Train: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/1/train.txt
2019-12-24 18:38:01,637 Dev: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/1/dev.txt
2019-12-24 18:38:01,641 Test: /content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/1/test.txt
Creating Label Dictionary ...
2019-12-24 18:38:06,633 Computing label dictionary. Progress:


100%|██████████| 46274/46274 [01:40<00:00, 462.60it/s]

2019-12-24 18:39:46,787 [b'matlab', b'c', b'c++', b'django', b'python', b'qt', b'r', b'embedded', b'machine-learning', b'flask']


Obtaining Corpus Statisitics...
{'TRAIN': {'dataset': 'TRAIN', 'total_number_of_documents': 46274, 'number_of_documents_per_class': {'matlab': 5229, 'c': 8166, 'c++': 10411, 'django': 10270, 'python': 12633, 'qt': 4285, 'r': 5522, 'embedded': 477, 'machine-learning': 972, 'flask': 1123}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 7790097, 'min': 9, 'max': 5701, 'avg': 168.34717119764878}}, 'TEST': {'dataset': 'TEST', 'total_number_of_documents': 5785, 'number_of_documents_per_class': {'python': 1630, 'c++': 1296, 'c': 1039, 'matlab': 611, 'machine-learning': 131, 'django': 1288, 'flask': 134, 'r': 687, 'qt': 546, 'embedded': 54}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 971689, 'min': 12, 'max': 3107, 'avg': 167.96698357821953}}, 'DEV': {'dataset': 'DEV', 'total_number_of_documents': 5784, 'number_of_documents_per_class': {'django': 1260, 'python': 1524, 'c++': 1337, 'matlab': 655, 'c': 1005, 'r': 694, 'qt': 553, 'embedded': 60, 'machine-learning

100%|██████████| 98656/98656 [03:15<00:00, 503.48it/s]

...

2019-12-24 19:45:16,100 [b'perl', b'regex']


Obtaining Corpus Statisitics...
{'TRAIN': {'dataset': 'TRAIN', 'total_number_of_documents': 7697, 'number_of_documents_per_class': {'perl': 4114, 'regex': 4086}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 1051398, 'min': 11, 'max': 2354, 'avg': 136.59841496687022}}, 'TEST': {'dataset': 'TEST', 'total_number_of_documents': 963, 'number_of_documents_per_class': {'perl': 502, 'regex': 524}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 133909, 'min': 12, 'max': 2886, 'avg': 139.0539979231568}}, 'DEV': {'dataset': 'DEV', 'total_number_of_documents': 962, 'number_of_documents_per_class': {'regex': 512, 'perl': 506}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 135990, 'min': 15, 'max': 960, 'avg': 141.36174636174636}}}
Creating Dumps ... 
Group ID being Processed : 14
/content/drive/My Drive/ICDMAI_Tutorial/notebook/training_data/normalised_training_data/group/14/
Creating Corpus ...
2019-12-24 19:45:52,317 Reading data from /content/drive

100%|██████████| 8819/8819 [00:19<00:00, 444.20it/s]

2019-12-24 19:46:14,605 [b'excel', b'excel-vba', b'vba']


Obtaining Corpus Statisitics...
{'TRAIN': {'dataset': 'TRAIN', 'total_number_of_documents': 8819, 'number_of_documents_per_class': {'excel': 5705, 'excel-vba': 4165, 'vba': 5521}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 1601143, 'min': 14, 'max': 4458, 'avg': 181.55607211702008}}, 'TEST': {'dataset': 'TEST', 'total_number_of_documents': 1103, 'number_of_documents_per_class': {'vba': 688, 'excel-vba': 526, 'excel': 700}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 194187, 'min': 19, 'max': 1804, 'avg': 176.05349048050772}}, 'DEV': {'dataset': 'DEV', 'total_number_of_documents': 1102, 'number_of_documents_per_class': {'vba': 683, 'excel-vba': 512, 'excel': 712}, 'number_of_tokens_per_tag': {}, 'number_of_tokens': {'total': 207979, 'min': 20, 'max': 1795, 'avg': 188.72867513611615}}}
Creating Dumps ... 


# Visualise Label Distribution

In [0]:
import matplotlib.pyplot as plt 
%matplotlib inline
import pandas as pd 
import json

In [3]:
path

'/content/drive/My Drive/ICDMAI_Tutorial/notebook/'

In [0]:
base_path = 'training_data/normalised/group/'
for grp_id in range(1,15) :
  file_path = path + base_path + str(grp_id) + '/corpus_statistics.json'

  with open(file_path,'r') as f :
    data = json.load(f)
